In [2]:
import numpy as np
from PIL import Image
from dash import Dash, dcc, html, Input, Output, State
import plotly.graph_objects as go
from io import BytesIO
import requests
import os

# Function to convert RGB to Hex
def rgb_to_hex(r, g, b):
    return f"#{r:02X}{g:02X}{b:02X}"

# Function to load an image (local or from URL)
def load_image(input_path):
    if input_path.startswith("http://") or input_path.startswith("https://"):
        response = requests.get(input_path)
        image = Image.open(BytesIO(response.content))
    else:
        if os.path.exists(input_path):
            image = Image.open(input_path)
        else:
            raise FileNotFoundError(f"The file at {input_path} does not exist.")
    return image

# Create Dash app
app = Dash(__name__)

# Load and process the image
image_path = input("Enter the image path or URL: ").strip()
image = load_image(image_path)
image = image.resize((100, 100))  # Resize for simplicity
image_array = np.array(image)

# Extract RGB values and pixel positions
height, width, _ = image_array.shape
grid_x, grid_y, grid_colors, grid_text = [], [], [], []
pixels_rgb = []

for y in range(height):
    for x in range(width):
        r, g, b = image_array[y, x]
        grid_x.append(x)
        grid_y.append(height - y - 1)  # Flip Y for visualization
        grid_colors.append(f"rgb({r},{g},{b})")
        grid_text.append(f"RGB: ({r},{g},{b})<br>Hex: {rgb_to_hex(r, g, b)}")
        pixels_rgb.append((r, g, b))

r_vals = [rgb[0] for rgb in pixels_rgb]
g_vals = [rgb[1] for rgb in pixels_rgb]
b_vals = [rgb[2] for rgb in pixels_rgb]

# Create initial plots for 2D image and 3D RGB cube
fig_2d = go.Figure()
fig_2d.add_trace(go.Scatter(
    x=grid_x, y=grid_y,
    mode="markers",
    marker=dict(size=4, color=grid_colors),
    hoverinfo="text",
    text=grid_text,
    showlegend=False,  # Remove legend
))

fig_2d.update_layout(
    title="2D Image with RGB Values",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False, scaleanchor="x", scaleratio=1),
    margin=dict(l=0, r=0, t=40, b=0),
    hovermode="closest",
)

fig_3d = go.Figure()
fig_3d.add_trace(go.Scatter3d(
    x=r_vals, y=g_vals, z=b_vals,
    mode='markers',
    marker=dict(
        size=3,
        color=[f'rgb({r},{g},{b})' for r, g, b in zip(r_vals, g_vals, b_vals)],
        opacity=0.8
    ),
    hoverinfo="text",
    text=[f"RGB: ({r},{g},{b})<br>Hex: {rgb_to_hex(r, g, b)}" for r, g, b in zip(r_vals, g_vals, b_vals)],
    showlegend=False,  # Remove legend
))

fig_3d.update_layout(
    title="3D RGB Cube",
    scene=dict(
        xaxis_title="Red",
        yaxis_title="Green",
        zaxis_title="Blue",
        xaxis=dict(range=[0, 255]),
        yaxis=dict(range=[0, 255]),
        zaxis=dict(range=[0, 255]),
    ),
    margin=dict(l=0, r=0, t=40, b=0),
)

# Add placeholders for dynamic highlights without extra text
highlight_2d = go.Scatter(
    x=[], y=[],
    mode="markers",
    marker=dict(size=10, color="black"),
    hoverinfo="skip",
    showlegend=False  # Remove legend
)

highlight_3d = go.Scatter3d(
    x=[], y=[], z=[],
    mode="markers",
    marker=dict(size=6, color="black"),
    hoverinfo="skip",
    showlegend=False  # Remove legend
)

fig_2d.add_trace(highlight_2d)
fig_3d.add_trace(highlight_3d)

# Layout for Dash app
app.layout = html.Div([
    # Main content area divided into 3 parts
    html.Div([
        # Left: 2D image plot with RGB values
        html.Div([
            dcc.Graph(id="2d-plot", figure=fig_2d, style={"width": "100%", "display": "inline-block"})
        ], style={"width": "33%", "display": "inline-block"}),

        # Middle: Highlighted RGB sample (color block)
        html.Div([
            html.Div(id="highlighted-details", style={
                "textAlign": "center", "marginTop": "20px", "fontSize": "14px", "width": "100%"})
        ], style={"width": "33%", "display": "inline-block"}),

        # Right: 3D RGB cube plot
        html.Div([
            dcc.Graph(id="3d-plot", figure=fig_3d, style={"width": "100%", "display": "inline-block"})
        ], style={"width": "33%", "display": "inline-block"})
    ], style={"display": "flex", "justifyContent": "space-between", "width": "100%"})
])

# Callback to update highlights and display details
@app.callback(
    [Output("2d-plot", "figure"), Output("3d-plot", "figure"), Output("highlighted-details", "children")],
    Input("2d-plot", "clickData"),
    [State("2d-plot", "figure"), State("3d-plot", "figure")],
)
def highlight_pixel(click_data, figure_2d, figure_3d):
    if click_data and "points" in click_data:
        point_idx = click_data["points"][0]["pointIndex"]
        x, y, (r, g, b) = grid_x[point_idx], grid_y[point_idx], pixels_rgb[point_idx]
        hex_color = rgb_to_hex(r, g, b)

        # Update 2D highlight position
        figure_2d["data"][-1]["x"] = [x]
        figure_2d["data"][-1]["y"] = [y]

        # Update 3D highlight position
        figure_3d["data"][-1]["x"] = [r]
        figure_3d["data"][-1]["y"] = [g]
        figure_3d["data"][-1]["z"] = [b]

        # Display details of the highlighted pixel (without extra labels)
        details = html.Div([
            html.Div(f"Position: (X: {x}, Y: {y})"),
            html.Div(f"RGB: ({r}, {g}, {b})"),
            html.Div(f"Hex: {hex_color}"),
            html.Div(
                style={
                    "width": "50px", "height": "50px", "backgroundColor": hex_color,
                    "margin": "10px auto", "border": "1px solid black"
                }
            )
        ])
        return figure_2d, figure_3d, details

    return figure_2d, figure_3d, "Click on a pixel to highlight it and see details."

if __name__ == "__main__":
    app.run_server(debug=True)
